In [4]:
import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt
from functools import partial

%matplotlib inline

from sklearn.model_selection import KFold
from sklearn.cross_validation import cross_val_score, cross_val_predict
from sklearn.pipeline import Pipeline

In [5]:
order_products_train_df = pd.read_csv('order_products__train.csv', dtype={'order_id': np.int32, 'product_id': np.int32,'add_to_cart_order': np.int16, 'reordered': np.int8})
order_products_prior_df = pd.read_csv('order_products__prior.csv')
orders_df = pd.read_csv('orders.csv')
products_df = pd.read_csv('products.csv')
aisles_df = pd.read_csv('aisles.csv')
departments_df = pd.read_csv('departments.csv')

In [6]:
# Merge DF's
items = pd.merge(products_df, departments_df, how='left')

In [7]:
items = pd.merge(items, aisles_df, how='left')

In [8]:
items.product_name = items.product_name.str.replace(' ', '_').str.lower()

In [9]:
items.dtypes

product_id        int64
product_name     object
aisle_id          int64
department_id     int64
department       object
aisle            object
dtype: object

In [10]:
# Memory Reduction
items.product_id = items.product_id.astype(np.int32)
items.aisle_id = items.aisle_id.astype(np.int32)
items.department_id = items.department_id.astype(np.int32)

In [11]:
op_train = pd.read_csv('order_products__train.csv',
                                     dtype={'order_id': np.int32, 'product_id': np.int32,
                                           'add_to_cart_order': np.int16, 'reordered': np.int8})

In [12]:
op_prior = pd.read_csv('order_products__prior.csv',
                                     dtype={'order_id': np.int32, 'product_id': np.int32,
                                           'add_to_cart_order': np.int16, 'reordered': np.int8})

In [13]:
orders = pd.read_csv('orders.csv', dtype={'order_id': np.int32, 
                                                           'user_id': np.int32, 
                                                           'order_number': np.int16,  # max 100, could use int8
                                                           'order_dow': np.int8, 
                                                           'order_hour_of_day': np.int8, 
                                                           'days_since_prior_order': np.float16})


In [14]:
orders.eval_set = orders.eval_set.replace({'prior': 0, 'train': 1, 'test':2}).astype(np.int8)
orders.days_since_prior_order = orders.days_since_prior_order.fillna(-1).astype(np.int8)

In [15]:
indexes = np.linspace(0, len(op_prior), num=10, dtype=np.int32)

train_details = pd.merge(op_train, orders, how='left', on='order_id').apply(partial(pd.to_numeric, errors='ignore'
                                                                                   ,downcast='integer'))

In [16]:
train_details = pd.merge(train_details, items[['product_id', 'aisle_id', 'department_id']].apply(partial(pd.to_numeric,
                                                                                                        errors='ignore',
                                                                                                        downcast='integer')),
                         how='left', on='product_id')

In [17]:
print(train_details.shape, op_train.shape)

(1384617, 12) (1384617, 4)


In [18]:
order_details = pd.merge(left=pd.merge(op_prior, items[['product_id', 'aisle_id', 'department_id' ]],
                                how='left', on='product_id'), right=orders, how='left', on='order_id')

In [19]:
#  Set eval_set to 1 because it is in the training set
train_orders = orders[orders.eval_set == 1]

In [20]:
train_orders.index.name = 'raw_order'
train_orders.reset_index(inplace=True)

......Ignore until xgboost is figured out......

In [21]:
import xgboost as xgb
from sklearn.grid_search import GridSearchCV

/Users/benjaminseifert/anaconda/lib/python3.6/site-packages/sklearn/grid_search.py:43: DeprecationWarning: This module was deprecated in version 0.18 in favor of the model_selection module into which all the refactored classes and functions are moved. This module will be removed in 0.20.
  DeprecationWarning)


In [54]:
cv_params = {'max_depth': [3,5,7], 'min_child_weight': [1,3,5]}
ind_params = {'learning_rate': 0.1, 'n_estimators': 1000, 'seed':0, 'subsample': 0.8, 'colsample_bytree': 0.5, 
             'objective': 'reg:logistic'}
optimized_GBM = GridSearchCV(xgb.XGBClassifier(**ind_params), 
                            cv_params, 
                             scoring = 'accuracy', cv = 5, n_jobs = -1) 

In [61]:
optimized_GBM.fit(y, X)

JoblibValueError: JoblibValueError
___________________________________________________________________________
Multiprocessing exception:
...........................................................................
/Users/benjaminseifert/anaconda/lib/python3.6/runpy.py in _run_module_as_main(mod_name='ipykernel_launcher', alter_argv=1)
    188         sys.exit(msg)
    189     main_globals = sys.modules["__main__"].__dict__
    190     if alter_argv:
    191         sys.argv[0] = mod_spec.origin
    192     return _run_code(code, main_globals, None,
--> 193                      "__main__", mod_spec)
        mod_spec = ModuleSpec(name='ipykernel_launcher', loader=<_f...b/python3.6/site-packages/ipykernel_launcher.py')
    194 
    195 def run_module(mod_name, init_globals=None,
    196                run_name=None, alter_sys=False):
    197     """Execute a module's code without importing it

...........................................................................
/Users/benjaminseife

...................................................................................

In [34]:
order_details.dtypes

order_id                  int32
product_id                int32
add_to_cart_order         int16
reordered                  int8
aisle_id                  int32
department_id             int32
user_id                   int32
eval_set                   int8
order_number              int16
order_dow                  int8
order_hour_of_day          int8
days_since_prior_order     int8
dtype: object

In [49]:
order_details.head()

,order_id,product_id,add_to_cart_order,reordered,aisle_id,department_id,user_id,eval_set,order_number,order_dow,order_hour_of_day,days_since_prior_order
0,2,33120,1,1,86,16,202279,0,3,5,9,8
1,2,28985,2,1,83,4,202279,0,3,5,9,8
2,2,9327,3,0,104,13,202279,0,3,5,9,8
3,2,45918,4,1,19,13,202279,0,3,5,9,8
4,2,30035,5,0,17,13,202279,0,3,5,9,8


In [48]:
train_orders.head()

,raw_order,order_id,user_id,eval_set,order_number,order_dow,order_hour_of_day,days_since_prior_order
0,10,1187899,1,1,11,4,8,14
1,25,1492625,2,1,15,1,11,30
2,49,2196797,5,1,5,0,11,6
3,74,525192,7,1,21,2,11,6
4,78,880375,8,1,4,1,14,10


In [24]:
order_details.head(50)

,order_id,product_id,add_to_cart_order,reordered,aisle_id,department_id,user_id,eval_set,order_number,order_dow,order_hour_of_day,days_since_prior_order
0,2,33120,1,1,86,16,202279,0,3,5,9,8
1,2,28985,2,1,83,4,202279,0,3,5,9,8
2,2,9327,3,0,104,13,202279,0,3,5,9,8
3,2,45918,4,1,19,13,202279,0,3,5,9,8
4,2,30035,5,0,17,13,202279,0,3,5,9,8
5,2,17794,6,1,83,4,202279,0,3,5,9,8
6,2,40141,7,1,105,13,202279,0,3,5,9,8
7,2,1819,8,1,88,13,202279,0,3,5,9,8
8,2,43668,9,0,123,4,202279,0,3,5,9,8
9,3,33754,1,1,120,16,205970,0,16,5,17,12


In [22]:
df = order_details.groupby('user_id')['order_number'].max()

In [26]:
df

user_id
1         10
2         14
3         12
4          5
5          4
6          3
7         20
8          3
9          3
10         5
11         7
12         5
13        12
14        13
15        22
16         6
17        40
18         6
19         9
20         4
21        33
22        15
23         4
24        18
25         3
26        12
27        81
28        24
29        18
30         8
          ..
206180    18
206181    14
206182    10
206183    14
206184     4
206185    10
206186     3
206187    34
206188     7
206189     6
206190     8
206191     5
206192    14
206193    41
206194    11
206195    19
206196     4
206197    23
206198     7
206199    19
206200    23
206201    32
206202    22
206203     5
206204     4
206205     3
206206    67
206207    16
206208    49
206209    13
Name: order_number, Length: 206209, dtype: int16

In [23]:
def order_retrieval(test_set):
    #     Match the user_id in training set to the user_id in test set
    test_set_prior = order_details[(order_details.user_id.isin(test_set.user_id))]
    #     Order by each user's biggest order (these will have the best learning for predicitons)
    test_mst_recent = test_set_prior.groupby('user_id')['order_number'].max()
    
    #   merge the test_mst_recent to test history where the user reordered 
    order_history = pd.merge(test_mst_recent.reset_index(), test_set_prior[test_set_prior.reordered == 1], 
                             how='left', on=['user_id', 'order_number'])

    #     merge order history (above) with the test orders to create a predicting model.. fill the null with -1. This indicates if the item predicted did not match the purchased
    test_set_prior = pd.merge(order_history[['user_id', 'product_id']], 
                              test_set[['user_id', 'order_id']], how='left', 
                              on='user_id').fillna(-1).groupby('order_id')['product_id'].apply(lambda x: ' '.join([str(int(e)) for e in set(x)])).reset_index().replace(to_replace='-1', value='None')
    test_set_prior.columns = ['order_id', 'products']
    
    # occasionally there is a bug where a line with order_id == -1 makes it through. doesn't *seem* to effect things
    return test_set_prior[test_set_prior.order_id > 0].set_index('order_id')

In [24]:
cvpreds = []

kf = KFold(3, shuffle=False, random_state=3)
for train_index, test_index in kf.split(train_orders.index):
    cvpreds.append(order_retrieval(train_orders.iloc[test_index]))
    
df_cvpreds = pd.concat(cvpreds).sort_index()
df_cvpreds.head()

,products
order_id,
1,43633 30881 5707 14947
36,35939 24964 26629 581 44359 47734 16759
38,33731 8012
96,24489 27966
98,4357 43654 34065 19731 1939 45204 33686 40986 ...


# Generate a CSV Submission Format F1 Score

In [ ]:
test = pd.read_csv('sample_submission.csv')

In [25]:
try:
    df_train = pd.read_csv('train.csv', index_col='order_id')
except:
    true_positives = []

    for uid, val in train_details.groupby('user_id'):
        val_1 = subset[subset.reordered == 1]
        oid = subset.order_id.values[0]

        if len(val_1) == 0:
            true_positives.append((oid, 'None'))
            continue

        ostr = ' '.join([str(int(e)) for e in val_1.product_id.values])
        # .strip is needed because join can have a padding space at the end
        true_positives.append((oid, ostr.strip()))

    df_train = pd.DataFrame(true_positives)

    df_train.columns = ['order_id', 'products']
    df_train.set_index('order_id', inplace=True)
    df_train.sort_index(inplace=True)
    
#     df_train.to_csv('train.csv')

In [28]:
f1_scores = []
for fp, pred in zip(df_train.sort_index().products, df_cvpreds.sort_index().products):
    fn_fp = fp.replace("None", "-1").split(' ')
    fn_pred = pred.replace("None", "-1").split(' ')
    
    r2 = (np.intersect1d(fn_fp, fn_pred))
    precision = np.float(len(r2)) / len(fn_pred)
    recall = np.float(len(r2)) / len(fn_fp)

    denom = precision + recall
    f1_scores.append(((2 * precision * recall) / denom) if denom > 0 else 0)

print(np.mean(f1_scores))

0.32558651711
